In [1]:
import imperial_materials_simulation as ims
import line_profiler as lp

# Profiling Tests

This code shows which lines of the source code take the longest to run.

Exact run times vary strongly from machine to machine.

In [2]:
sim = ims.Simulation(n_atoms=21, starting_temperature=800, microstructure=ims.starting_microstructure)
#running quick sims to compile physics backend for more consistent benchmarking
sim.NVT_run(n_steps=1_000, temperature=500)
sim.MMC_run(n_steps=1_000, temperature=500)

### NVT Run

In [3]:
profiler = lp.LineProfiler()
profiled_func = profiler(sim.NVT_run)
profiled_func(n_steps=100_000, temperature=1200, gamma=0.005, integrator='OVRVO')

In [4]:
profiler.print_stats(output_unit=1e-3) #ms times displayed

Timer unit: 0.001 s

Total time: 7.92132 s
File: C:\Users\Ayham\Documents\imperial-materials-simulation\src\imperial_materials_simulation\main.py
Function: NVT_run at line 131

Line #      Hits         Time  Per Hit   % Time  Line Contents
   131                                              def NVT_run(self, n_steps: int, temperature: float, gamma: float = 0.005, integrator: str = 'OVRVO'):
   132                                                 '''
   133                                                 Constant number of atoms, volume, and temperature (NVT) simulation with a Langevin thermostat. Simple
   134                                                 model that approximates the physical influence of a solute bath at a given temperature. It adds a drag
   135                                                 and random force onto the energy minimization force.
   136                                           
   137                                                 Parameters
   138  

### Relax Run

In [5]:
profiler = lp.LineProfiler()
profiled_func = profiler(sim.relax_run)
profiled_func(n_steps=100_000, step_size=0.03)

In [6]:
profiler.print_stats(output_unit=1e-3) #ms times displayed

Timer unit: 0.001 s

Total time: 5.79199 s
File: C:\Users\Ayham\Documents\imperial-materials-simulation\src\imperial_materials_simulation\main.py
Function: relax_run at line 93

Line #      Hits         Time  Per Hit   % Time  Line Contents
    93                                              def relax_run(self, n_steps: int, step_size: float = 0.01) -> None:
    94                                                 '''
    95                                                 Structural relaxation through steepest descent energy minimisation. This is a non-physical simulation
    96                                                 where the molecule is modelled as not experiencing any thermal forces. Each atom moves in the direction
    97                                                 that minimizes is potential energy the most, which is the net (bonding + non-bonding) force vector. 
    98                                           
    99                                                 Par

### MMC Run

In [7]:
profiler = lp.LineProfiler()
profiled_func = profiler(sim.MMC_run)
profiled_func(n_steps=200_000, temperature=800, random_scale=0.1)

In [8]:
profiler.print_stats(output_unit=1e-3) #ms times displayed

Timer unit: 0.001 s

Total time: 5.19868 s
File: C:\Users\Ayham\Documents\imperial-materials-simulation\src\imperial_materials_simulation\main.py
Function: MMC_run at line 273

Line #      Hits         Time  Per Hit   % Time  Line Contents
   273                                              def MMC_run(self, n_steps: int, temperature: float, random_scale: float = 0.05):
   274                                                 '''
   275                                                 Metropolis Monte Carlo (MMC) Simulation. Randomly displaces atoms by a small amount and accepts the new
   276                                                 structure if it reduces total potential energy or has a chance of accepting the new structure ∝ exp(-ΔPE)
   277                                                 if it increases total potential energy. Un-physical but useful for sampling a variety of fairly low energy
   278                                                 microstructures for thermodynam

### NVE Run

In [9]:
profiler = lp.LineProfiler()
profiled_func = profiler(sim.NVE_run)
profiled_func(n_steps=100_000, temperature=600, gamma=0.005, integrator='Verlet')

In [10]:
profiler.print_stats(output_unit=1e-3) #ms times displayed

Timer unit: 0.001 s

Total time: 6.30773 s
File: C:\Users\Ayham\Documents\imperial-materials-simulation\src\imperial_materials_simulation\main.py
Function: NVE_run at line 200

Line #      Hits         Time  Per Hit   % Time  Line Contents
   200                                              def NVE_run(self, n_steps: int, temperature: float = None, gamma: float = 0.005, integrator: str = 'OVRVO'):
   201                                                 '''
   202                                                 Constant number of atoms, volume, and energy (NVE) simulation. Used for modelling molecules in the
   203                                                 gas phase or finding an acceptable simulation timestep. Larger timesteps speed up the simulation
   204                                                 but will eventually break conservation of energy if too large. https://doi.org/10.1021/jp411770f
   205                                                 details different technique